# **K-Means aplicado a texto**

El presente documento busca entender como funciona el algoritmo K-means en el análisis de texto, en el marco de estudio del procesamiento del lenguaje natural. 

Tomamos como base un set de datos de clasificación de peliculas con el objetivo de subclasificar las mismas en mas opciones que "positiva" o "negativa" 

## **Preparación de datos**

### **Carga de datos**

Para el uso practico y para lograr entender como funciona limitamos el dataset a la solo las primeras 100 filas. 

Como vamos a modificar la columna de interes creamos una copia para despues poder estudiar en profundidad el resultado. 

Mostramos los primeros 2 solo para ver como va quedando. 

In [ ]:
import pandas as pd 

datos = pd.read_csv("/kaggle/input/imdb-dataset-of-50k-movie-reviews-spanish/IMDB Dataset SPANISH.csv")
datos = datos.head(100)
datos["review_es_original"] = datos["review_es"]
datos.head(2)

### **Normalizar**

Antes de procesar el texto convertir a minúsculas, eliminar acentos, etc. Esto para garantizar que palabras iguales que estén escritas de distintas formas se tomen como diferentes. 

En este caso usaremos la libreria unicodedata. 

**Por ejemplo, observemos el siguiente texto:**


In [ ]:
print(datos["review_es"][1])

**Aplicamos la normalización en 2 partes:**

* Normalización ‘NFD’ (Normalization Form D). Esta forma descompone los caracteres acentuados en sus componentes básicos. Por ejemplo, la letra “á” se descompone en “a” y el acento (En este paso incluimos la función de pandas para pasar todo a minuscula).
* Encode ('ascii', 'ignore'): Convierte cada cadena de texto a bytes ASCII, ignorando los caracteres que no pueden ser convertidos a ASCII. Esto elimina los caracteres acentuados y otros caracteres especiales.

Luego Decode('utf-8') Convierte los bytes de vuelta a una cadena de texto en formato UTF-8.

> Nota: Se tiene que usar lamda dado que no podemos aplicar la normalización a la serie de datos completa, hay que hacerlo por cada elemento. 


In [ ]:
import unicodedata

datos["review_es"] = datos["review_es"].apply(lambda x: unicodedata.normalize('NFD', x).lower())
datos["review_es"] = datos["review_es"].apply(lambda x: x.encode('ascii', 'ignore').decode('utf-8'))

**Como podemos observar se aplicaron las correcciones al texto:**

In [ ]:
print(datos["review_es"][1])

### **Stop Words**

Las stop words (o palabras vacías) son palabras que, por sí solas, no aportan mucho significado y suelen ser ignoradas por los motores de búsqueda al indexar contenido. Estas palabras incluyen artículos, preposiciones, conjunciones y pronombres, entre otras. El objetivo de ignorarlas es centrarse en las palabras clave más relevantes para mejorar la eficiencia en la búsqueda y el análisis de texto.

Lista de stop words comunes en español
Algunas stop words comunes en español incluyen:

* Artículos: el, la, los, las, un, una, unos, unas
* Preposiciones: a, ante, bajo, con, contra, de, desde, en, entre, hacia, hasta, para, por, según, sin, sobre, tras
* Conjunciones: y, o, pero, aunque, sino, que
* Pronombres: yo, tú, él, ella, nosotros, vosotros, ellos, ellas, me, te, se, nos, os, le, les

> Nota: Para este caso practico al ser reseñas de peliculas se agregó a stopwords pelicula y escena. 

*Usamos la libreria nltk que ya tiene una lista de stopwords en español:*

In [ ]:
import nltk
from nltk.corpus import stopwords
stop_words_es = stopwords.words('spanish')
stop_words_es.extend(["pelicula", "peliculas", "escena", "escenas"])
print(f"{stop_words_es[0:5]}, entre otras.")

## **Vectorizamos el texto** 

Se debe pasar el texto a valores numericos: para esto se convierte todo el set (tomamos la columna review_es) en una matriz, donde cada fila va a ser una reseña y cada columna una palabra. La matriz va a alojar la cantidad de veces que la palabra se repite en el texto.

Pasamos como parámetro las stopwords que definimos en el paso anterior para que no las tenga en cuenta. 

> Nota: el resultado es una matriz dispersa.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(stop_words=stop_words_es)
matriz_textos = vectorizer.fit_transform(datos["review_es"])

matriz_textos

### Explicación: Matriz dispersa (sparse matrix)

Es un tipo de matriz en la que la mayoría de sus elementos son cero. A diferencia de una matriz densa, donde la mayoría de los elementos tienen valores distintos de cero, las matrices dispersas son comunes en situaciones donde los datos son escasos o tienen muchos valores nulos.

Compressed Sparse Row (CSR): En lugar de almacenar todos los elementos de la matriz (incluyendo los ceros), CSR solo almacena los valores distintos de cero junto con información adicional para reconstruir la matriz completa cuando sea necesario. Esto permite ahorrar una gran cantidad de memoria, especialmente cuando la matriz es muy grande y dispersa.

El resultado tiene 10288 elementos distintos de cero, 100 filas y 5206 columnas.

## **Para entender el resultado**

Solo para entender como funciona... bajamos la matriz a un sheet de excel. Para esto debemos pasarla de una matriz densa a una matriz común. 

In [ ]:
matriz_densa = matriz_textos.toarray()
nombres_columnas = vectorizer.get_feature_names_out()
df = pd.DataFrame(matriz_densa, columns=nombres_columnas)
df.to_excel("mi_matriz_de_textos.xlsx", index=False) 

### Por ejemplo: 

Del análisis del excel podemos observar que para el indice cero que corresponde a la primer reseña, la palabra violencia se repite 4 veces.

Para mostrarlo filtramos de la matriz:

In [ ]:
df.iloc[0:10,5090:6000]

**Si observamos la primer reseña esto es efectivamente correcto** 

"Uno de los otros críticos ha mencionado que después de ver solo 1 Oz Episodio, estará enganchado. Tienen razón, ya que esto es exactamente lo que sucedió conmigo. La primera cosa que me golpeó sobre Oz fue su brutalidad y sus escenas de **violencia** inconfiadas, que se encuentran a la derecha de la palabra. Confía en mí, este no es un espectáculo para los débiles de corazón o tímido. Este espectáculo no extrae punzones con respecto a las drogas, el sexo o la **violencia**. Es Hardcore, en el uso clásico de la palabra. Se llama OZ, ya que es el apodo dado al Penitenciario del Estado de Seguridad Máximo de Oswald. Se centra principalmente en la ciudad de Emeralda, una sección experimental de la prisión donde todas las células tienen frentes de vidrio y se enfrentan hacia adentro, por lo que la privacidad no es alta en la agenda. Em City es el hogar de muchos ... Fariarios, musulmanes, gangstas, latinos, cristianos, italianos, irlandeses y más ... así que las esposas, las miradas de muerte, las relaciones peligrosas y los acuerdos sombreados nunca están lejos. Yo diría el principal atractivo de El espectáculo se debe al hecho de que va donde otros espectáculos no se atreverían. Olvídate de las imágenes bonitas pintadas para las audiencias convencionales, olvidan el encanto, olviden el romance ... Oz no se mete. El primer episodio que he visto me sorprendió tan desagradable que fue surrealista, no podía decir que estaba listo para ello, pero cuando observé más, desarrollé un gusto por Oz, y me acostumbré a los altos niveles de **violencia** gráfica. No solo la **violencia**, sino la injusticia (Guardias torcidas que se vendrán por un níquel, los reclusos que se matarán en orden y se alejarán con él, de manera educada, los reclusos de clase media se convirtieron en perras de la prisión debido a su falta de habilidades callejeras O experiencia en la prisión) viendo oz, puede sentirse cómodo con lo que es incómodo visualización ... eso es si puedes ponerte en contacto con tu lado más oscuro."

## **Creamos un modelo K-Means**

Importamos K-Means desde Skitlearn

Creamos el modelo y le indicamos la cantidad de cluster objetivo. El valor n_init establece cuantas veces se recalculan los centroides.

Luego ejecutamos el modelo con fit_predict, tomando como referencia la matriz de texto. 

In [ ]:
from sklearn.cluster import KMeans

modelo = KMeans(n_clusters=6, n_init=10)
predicciones = modelo.fit_predict(matriz_textos)

## **Aplicamos las predicciones al set de datos**

Creamos una columna nueva en el dataset original y le aplicamos las predicciones

In [ ]:
datos["clasifica"] = predicciones
datos.to_excel("resultado.xlsx")
datos.head()

# **Conclusión** 

En relación a la versión 1, incluyendo un filtro para las *(StopWords)* el desempeño del algoritmo mejoró bastante. 

En el ejemplo a continuación vemos como se han clasificado todas las reseñas que poseen la palabra "Violencia". Sin profundizar en el análisis en relación a la primer ejecución observamos que la mayoria de las reseñas con la palabra violencia se clasificaron en el mismo cluster. 

> Nota: Mas abajo se muestra la nube de palabras de dicho cluster.

In [ ]:
df_filtrado = datos[datos['review_es'].str.contains('violencia')]
df_filtrado

### **Nube de palabras sin aplicar stopwords**

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

texto = " ".join(datos["review_es"].astype(str))
 
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(texto)

# Visualizar el wordcloud
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()

### **Nube de palabras luego de aplicar stopwords**

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

texto = " ".join(datos["review_es"].astype(str))
 
wordcloud = WordCloud(width=800, height=400, background_color='white', stopwords=stop_words_es).generate(texto)

# Visualizar el wordcloud
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()

### **Nube de palabras solo del cluster 0**

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

cluster_reviews = datos[datos["clasifica"] == 0]["review_es"]

texto = " ".join(cluster_reviews.astype(str))
 
wordcloud = WordCloud(width=800, height=400, background_color='white', stopwords=stop_words_es).generate(texto)

# Visualizar el wordcloud
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()

### **Nube de palabras de reseñas que contienen la palabra violencia**

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

cluster_reviews = df_filtrado['review_es']

texto = " ".join(cluster_reviews.astype(str))
 
wordcloud = WordCloud(width=800, height=400, background_color='white', stopwords=stop_words_es).generate(texto)

# Visualizar el wordcloud
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()

# ***Para continuar investigando***

En el cluster cero como podemos observar se agruparon reseñas que principalmente contienen la palabra "Si", "Ma" y "Ver"... es esto util? 

*Las reseñas con la palabra violencia* fueron al mismo cluster... destacando las mismas palabras que las mencionadas anteriormente. 